# Lesson 2.1 Genetic Algorithms
---

## Introduction

Originally termed Machine Evolution, Genetic Algorithms are inspired by the process of natural selection (evolution). In genetic algorithms, successor states are generated by combining 2 parent states. This is analogous to Natural Selection by means of sexual reproduction  in nature. Here, as in nature, random mutations occur in the offspring. For each generation set, the best mate, reproduce, and create a new generation.

## The Algorithm

The Genetic algorithm has many variants, but at its core, it is the same process of mating, reproducing, mutating, and back again. Some variants will terminate after a specified number of generations have been generated or once an objective is met. Each entity in a genetic algorithm is represented as a binary string, known as a genetic string. This string encodes information of the entity in relation to the given problem. The process can be seen visually below:

![GA](./images/gadiagram.png)

Here, we start with a randomly generated population set of size *k*. This population is then evaluated and their fitness score is determined. The fitness score is calculated based on the constraints or objectives the algorithm intends to solve. Once the fitness scores have been determined, their probability of mating is determined. This is directly proportional to the fitness score. 

Next, in the selection stage, pairs are selected at random with accordance to their mating probabilities. There will be some that will not be chosen in the mating process, and others will have more than one mate.

The next stage is the crossover stage. Here, each pair will contain a randomly generated crossover point in their genetic string. At this crossover point, one offspring will contain the first part of one parent and the second part of the other parent, and another offspring will contain the reverse. 

The mutation stage is the process of taking the generated offspring and randomly selecting them with accordance of a mutation probability and mutating a random bit within their genetic string. Some mutations will not matter too much, and others will be a drastic change that will either be discarded because it can't survive under the given conditions, or it will change the direction of the population where the mutation will propogate through the population in successive generations.

Lastly, it starts over with the evaluation stage where the newly generated population is evaluated. As stated previously, this process will run continuously untill one of two desired outcomes: an objective has been met or the number of generations has reached a designated number.

### Pseudocode

In [1]:
#Arguments:
#    Population: Initial Population set
#    FitnessFunction: Function to evaluate the fitness of an individual
def GeneticAlgorithm(population, FitnessFuntion):
    while ~done:
        new_population = []
        for member in population:
            x = RandomSelection(population, FitnessFunction)
            y = RandomSelection(population, FitnessFunction)
            child = Reproduce(x,y)
            if DoMutate():
                Mutate(child)
            add_child(new_population,child)
        population = new_population

def Reproduce(x,y):
    n = length(x)
    c = GetRandom(1,n)
    return SubString(x,1,c) + SubString(y,c+1,n)
              
    
    

## Visual Example: 8 Queens Problem

The 8 queens problem involves an 8x8 chess board and 8 queens. The problem is to place all 8 queens on the board where no 2 queens can attack each other. If we were to attempt to solve this problem by means of trial and error, it would take some time to reach a solution as there are 4,426,165,368 possible arrangements of 8 queens on an 8x8 grid and only 92 possible valid solutions. Therefore, we will be using a genetic algorithm to search for the solution.

We will represent a generated solution by a genetic string. However, for simplicity, the string will consist of 8 numbers where the number index represents the column (left to right) and the number at the index represents the row (bottom to top). For example, take the following board arrangement:

![board1](./images/board1.png)

The above board would be described as the string: **86427531**. In our fitness function, we will use the number of non attacking pairs of queens. This results in a number of 28 for a solution. Therefore, a higher score means a better solution. After we calculate the fitness, we calculate the probabilty of a specific solution to mate given its fitness. In this solution set, our probability of any 2 solutions mating is about 36%. We then multiply this percentage by our fitness to obtain our fitness based probablity. This  As an example of what will be done, view the below image.

![process](./images/process.PNG)

Now that you have seen how it will be implemented, try the following example code:

In [2]:
import numpy as np
import ipywidgets as widgets
from GA import GeneticAlgorithm

#Create Widgets for simple use
PopulationSize = widgets.IntSlider(value=100,min=20,max=2000,step=1)
NumGenerations = widgets.IntSlider(value=1000,min=100,max=10000,step=1)
MutationProbability = widgets.IntSlider(value=20,min=0,max=100,step=1)
countWidget = widgets.IntProgress(value=0, description='Generation')
solutionWidget = widgets.Text(description='Best')
runButton = widgets.Button(description='Run Algorithm')

bx1 = widgets.HBox([widgets.Label('Population Size:'),PopulationSize])
bx2 = widgets.HBox([widgets.Label('Number of Generations:'),NumGenerations])
bx3 = widgets.HBox([widgets.Label('Mutation Probability:'),MutationProbability])

#Some Global Variables
GoalScore=28
NumQueens=8

#Function to Create Initial Population
def CreatePopulation():
    population=[]
    for i in range(0,PopulationSize.value):
        geneticString=""
        for j in range(0,NumQueens):
            geneticString += str(np.random.randint(1,NumQueens+1))
        population.append(geneticString)
    return population
#Function to evaluate the fitness of the population
def fitnessFunction(x):
    count=GoalScore
    for i in range(0,NumQueens):
        col1 = i
        row1 = int(x[i])
        for j in range(0,NumQueens):
            if j==i: 
                continue
            col2 = j
            row2 = int(x[j])
            if (row1==row2) or (abs(row1-row2) == abs(col1-col2)):
                count = count-1
    return (float(count)/float(GoalScore))

def RunGeneticAlgorithm(b):
    GA = GeneticAlgorithm(1,NumQueens+1,MutationProbability.value,CreatePopulation,fitnessFunction)
    SolutionFound=False
    countWidget.min=0
    countWidget.max = NumGenerations.value
    countWidget.value=0
    while GA.generation<NumGenerations.value and GA.converged<1.0:
        GA.NextGeneration()
        countWidget.value = GA.generation
        solutionWidget.value = str(GA.bestDna)
    countWidget.value = countWidget.max

#Display Widgets
runButton.on_click(RunGeneticAlgorithm)
widgets.VBox([bx1,bx2,bx3,runButton,countWidget,solutionWidget])

A Jupyter Widget

## Aeromechanical Example: Find Sensor Placement

In Aeromechanical testing, the first step is to find where to place the sensors. We do this by utilizing finite element models, where the complex model is broken up into smaller primitive types and the stress/strain of these elemnts are more easily computed. The streeses and strains for a given element are defined at the element's nodes. For simplicity, this example will only deal with placement at a given node, off-node mappings will not be performed.

Below, you will set the settings for the model. By specifying the number of nodes, the number of dynamic shapes, and the number of sensors, the format for the genetic string will be created. The genetic string will be a binary number specified as the following:

$$NumSensors * \log_{2} (NumNodes) = NumBits$$

Given the above, the genetic string will contain information for the node the sensor is mapped. Setting these values will also populate a data set. The data set will be generated by randomly selecting a node for each mode set, assigning the node a value of 1.0 representing 100% ratio to max and then all other nodes surrounding it will taper to 0. For example: Given a 5 node setting, node 3 is randomly selected. Node 3 ratio will be 1.0. Therefore, Node 2 = 0.5, Node 1 = 0.0, Node 4 = 0.5, Node 5 = 0.0.

After setting the model settings, you will need to set the settings for the genetic algorithm. This will consist of the same information as the previous example, but will add an objective. The objective function is set to the minimum amplitude ratio for the design to detect for all modes. For example, if 2 modes are set, one sensor, and an objective of 0.3 the mapped sensor's strain must ratio to the max strain of each mode >= 0.3.

In [4]:
import ipywidgets as widgets
import math
import numpy as np
import bisect
from GA import GeneticAlgorithm

#Model Settings
NumNodes = widgets.IntSlider(value=1000,min=1000,max=1000000,step=100)
NumDynamic = widgets.IntSlider(value=3,min=1,max=100,step=1)
NumSensors = widgets.IntSlider(value=2,min=1,max=10,step=1)
#Algorithm Settings
pSize = widgets.IntSlider(value=100,min=20,max=2000,step=1)
nGenerations = widgets.IntSlider(value=500,min=100,max=10000,step=1)
mProb = widgets.IntSlider(value=20,min=0,max=100,step=1)
Objective = widgets.FloatSlider(value=0.3,min=0.01,max=1.0,step=0.01)
countOut = widgets.IntProgress(value=0, description='Generation')
solutionOut = widgets.Text(description='Best')
coverageOut = widgets.Text(description='Coverage')
optimize = widgets.Button(description='Run Algorithm')

bx1 = widgets.HBox([widgets.Label('Number of Nodes'),NumNodes])
bx2 = widgets.HBox([widgets.Label('Number of Dynamic Modes:'),NumDynamic])
bx3 = widgets.HBox([widgets.Label('Number of Sensors'),NumSensors])
bx4 = widgets.HBox([widgets.Label('Population Size:'),pSize])
bx5 = widgets.HBox([widgets.Label('Number of Generations:'),nGenerations])
bx6 = widgets.HBox([widgets.Label('Mutation Probability:'),mProb])
bx7 = widgets.HBox([widgets.Label('Minimum Amplitude Ratio:'),Objective])

#Create the data set holding the strain ratio for each node for each dynamic mode shape
def CreateData():
    dataSet=[]
    numNodes = NumNodes.value
    resolution = 0.05
    numModes = NumDynamic.value
    for i in range(0,numModes):
        modeData = [0.0] * numNodes
        maxPoint = np.random.randint(0,numNodes)
        modeData[maxPoint] = 1.0
        cval = 1.0
        for j in range(0,maxPoint):
            if cval > 0: cval -= resolution
            else: cval=0
            modeData[maxPoint-j - 1] = cval
        cval = 1.0
        for j in range(maxPoint+1,numNodes):
            if cval > 0: cval -= resolution
            else: cval=0
            modeData[j] = cval
        dataSet.append(modeData)
    return dataSet

#Get Binary Representation of a number
def get_bin(x, n=0):
    return format(x, 'b').zfill(n)

#Get Integer from a binary string
def get_int(x):
    return int(x,2)

#Create the initial population
def CreateMappings():
    numSensors = NumSensors.value
    numNodes = NumNodes.value
    numSensorBits = math.ceil(math.log(numNodes)/math.log(2))
    population=[]
    for i in range(0,pSize.value):
        dna = ""
        for j in range(0,numSensors):
            dna += get_bin(np.random.randint(0,numNodes),numSensorBits)
        population.append(dna)
    return population

#Test the coverage of the current design. i.e What modes are covered
def GetCoverage(x):
    numSensors = NumSensors.value
    numNodes = NumNodes.value
    numModes = NumDynamic.value
    numSensorBits = math.ceil(math.log(numNodes)/math.log(2))
    coverage = [False]*numModes
    for i in range(0,numModes):
        for j in range(0,numSensors):
            sensor = x[(j*numSensorBits):((j*numSensorBits)+numSensorBits)]
            node = get_int(sensor)
            if node < numNodes and data[i][node]>=Objective.value:
                coverage[i] = True
    return coverage

#Fitness function
def TestMappings(x):
    numModes = NumDynamic.value
    coverage = GetCoverage(x)
    total=0
    for c in coverage:
        if c == True:
            total+=1
    total /= numModes

    return total

#Run the algorithm
def RunGeneticAlgorithm2(b):
    global data
    data = CreateData()
    GA = GeneticAlgorithm(0,1,mProb.value,CreateMappings,TestMappings)
    countOut.min=0
    countOut.max = nGenerations.value
    countOut.value=0
    while GA.generation<nGenerations.value and GA.converged<1.0:
        GA.NextGeneration()
        solutionOut.value = str(GA.bestDna)
        coverageOut.value = str(GetCoverage(GA.bestDna))
        countOut.value = GA.generation
    countOut.value = countOut.max

#Display Widgets
optimize.on_click(RunGeneticAlgorithm2)
widgets.VBox([widgets.HTML('<b>Model Settings</b>'),
              bx1,bx2,bx3,
              widgets.HTML('<b>Algorithm Settings</b>'),
              bx4,bx5,bx6,
              widgets.HTML('<b>Objective</b>'),
              bx7,
              widgets.Label(' '),
              optimize,solutionOut,coverageOut,countOut])

A Jupyter Widget

## Review

In this lesson we have gone over the genetic algorithm. We have discussed what the algorithm is and how it works. We have also gone over detailed examples of implementation. The first example sought to create a solution for the NQueens puzzle. We saw how the algorithm worked, saw the example implementation, and observed its performance. Finally, we looked at an example as it relates to the field of aeromechanical testing. This example sought to find a solution where the specified number of sensors were able to cover one or more dynamic mode shapes with a set minimum objective ratio. The genetic algorithm is a great tool and works well for state spaced models. Please review the sample code, try to improve it, and play with the available dials for tuning the algorithm. What are some of the things that are noticed when running the algorithm. Kepp an eye on the best DNA string and note what is observed.

The Class for the Genetic Algorith used in the examples can be found below:

[GA.py](./GA.py)

## Resources

Russel, S. J., & Norvig, P. (2015). Beyond Classical Search. In Artificial intelligence: A modern approach (3rd ed., pp. 127-130). Noida, India: Pearson India Education Services Pvt.

A. (2018, June 22). Aimacode/aima-python. Retrieved June 26, 2018, from https://github.com/aimacode/aima-python